In [154]:
import pandas as pd
import numpy as np
import spacy
import itertools
import pickle
import re
import json
import importlib

### take a look at data and save a small sample

In [126]:
train = json.load(open("./data/train-v2.0.json", "r"))["data"]

In [127]:
small_train = [i for i in train if i["title"] in ["Buddhism", "Space_Race"]]
tinny_train = []
for i in small_train:
    short_i = i
    short_i["paragraphs"] = i["paragraphs"][:3]
    tinny_train.append(short_i)

In [128]:
json.dump({"version": "v2.0", "data": tinny_train}, open("./data/tiny_train.json", "w"))

In [129]:
for title in tinny_train:
    print(title["title"])
    print("number of paragraphs:",len(title["paragraphs"]))
    for i, paragrah in enumerate(title["paragraphs"]):
        print("number of questions in paragraph {}: {}".format(i, len(paragrah["qas"])))

Buddhism
number of paragraphs: 3
number of questions in paragraph 0: 16
number of questions in paragraph 1: 10
number of questions in paragraph 2: 10
Space_Race
number of paragraphs: 3
number of questions in paragraph 0: 4
number of questions in paragraph 1: 2
number of questions in paragraph 2: 3


In [187]:
### look at dev data: clearly dev data has multiple ground truth answers, while training data has only one
dev = json.load(open("./data/dev-v2.0.json", "r"))["data"]

In [185]:
[i for i in dev if i ["title"] == "Oxygen"][0]["paragraphs"][0]["qas"][1:3]

[{'answers': [{'answer_start': 331, 'text': 'helium'},
   {'answer_start': 331, 'text': 'helium'},
   {'answer_start': 331, 'text': 'helium'},
   {'answer_start': 331, 'text': 'helium'},
   {'answer_start': 331, 'text': 'helium'}],
  'id': '571a484210f8ca1400304fbe',
  'is_impossible': False,
  'question': 'What is the second most abundant element?'},
 {'answers': [{'answer_start': 485, 'text': 'Diatomic oxygen'},
   {'answer_start': 485, 'text': 'Diatomic oxygen'},
   {'answer_start': 485, 'text': 'Diatomic oxygen gas'},
   {'answer_start': 485, 'text': 'Diatomic oxygen'},
   {'answer_start': 485, 'text': 'Diatomic oxygen gas'}],
  'id': '571a484210f8ca1400304fbf',
  'is_impossible': False,
  'question': "Which gas makes up 20.8% of the Earth's atmosphere?"}]

### Get a same small sample from v1.0

In [188]:
train1 = json.load(open("./data/train-v1.1.json", "r"))["data"]

In [189]:
small_train = [i for i in train1 if i["title"] in ["Buddhism", "Space_Race"]]
tinny_train = []
for i in small_train:
    short_i = i
    short_i["paragraphs"] = i["paragraphs"][:3]
    tinny_train.append(short_i)

In [193]:
json.dump({"version": "1.1", "data": tinny_train}, open("./data/tiny_train-v1.1.json", "w"))

### Test data_utils

In [130]:
importlib.reload(data_utils)
import data_utils

In [131]:
file_dict = {"train": "./data/tiny_train.json",
             "val": "./data/tiny_train.json"}

In [134]:
train_data = data_utils.prep_data(file_dict["train"])

In [147]:
len(train_data)

6

In [152]:
for batch in train_data:
    print("---Context\n"+batch.context)
    for i, (q,a) in enumerate(zip(batch.q, batch.a)):
        print("Q{}:{}".format(i, q))
        print("A{}:{}".format(i, a))
        print("id", batch.id[i])
        print("start index", batch.a_index[i])
    break

---Context
Buddhism /ˈbudɪzəm/ is a nontheistic religion[note 1] or philosophy (Sanskrit: धर्म dharma; Pali: धम्म dhamma) that encompasses a variety of traditions, beliefs and spiritual practices largely based on teachings attributed to Gautama Buddha, commonly known as the Buddha ("the awakened one"). According to Buddhist tradition, the Buddha lived and taught in the eastern part of the Indian subcontinent, present-day Nepal sometime between the 6th and 4th centuries BCE.[note 1] He is recognized by Buddhists as an awakened or enlightened teacher who shared his insights to help sentient beings end their suffering through the elimination of ignorance and craving. Buddhists believe that this is accomplished through the direct understanding and perception of dependent origination and the Four Noble Truths.
Q0:What type of religion is Buddhism?
A0:nontheistic
id 56cff91b234ae51400d9c1bb
start index 25
Q1:What are the practices of Buddhism based on?
A1:teachings attributed to Gautama Budd